# Actual Draft for the Script to run every Sunday

1. Get top 20 stocks by market cap 
2. Get historical returns for the last X days of those stocks 
3. Cumulate these returns and return a sorted dataframe
4. Gather the top and last 3 
5. Intersect with the attention data (will have to scrape that manually, every Sunday (monday?))

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import rpy2.robjects as robjects


import numpy as np

import datetime

***
## Universe Scraper 
Grab top 20 market cap cryptocurrencies (scraped from CoinMarketCap)

In [48]:
###=== UNIVERSE SCRAPER ===###

url = "https://coinmarketcap.com/"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table")

rows = table.find_all("tr")

ticker_list = []

for i, row in enumerate(rows[1:101]): # exclude header row
    if i < 10:
        ticker = row.find("a", {"class": "cmc-link"}).find('p', {'class': 'coin-item-symbol'}).text
    else:
        ticker = row.find("a", {"class": "cmc-link"}).find("span", {"class": "crypto-symbol"}).text

    ticker_list.append(ticker)

In [49]:
###=== CLEANING AND SELECTION OF THE UNIVERSE ===###

no_trades = ['USD', 'DAI', 'WBTC', 'WETH']

for i in range(len(no_trades)):    
    ticker_list = [x for x in ticker_list if no_trades[i] not in x]

top_fifteen = ticker_list[:15]

In [50]:
top_fifteen

['BTC',
 'ETH',
 'XRP',
 'BNB',
 'ADA',
 'SOL',
 'DOGE',
 'TRX',
 'MATIC',
 'LTC',
 'DOT',
 'AVAX',
 'BCH',
 'TON',
 'SHIB']

***
## Price Scraper

In [51]:
start_date = (datetime.date.today() - datetime.timedelta(21)).strftime('%d-%m-%Y')
end_date = (datetime.date.today()).strftime('%d-%m-%Y')

### With CMC Scraper

In [52]:
from cryptocmd import CmcScraper

# Inital DF with Bitcoin 
scraper = CmcScraper("BTC", start_date, end_date)
df = pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':'BTC'})

for crypto in top_twenty[1:]:
    scraper = CmcScraper(str(crypto), start_date, end_date)
    df = pd.merge(df, pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':str(crypto)}), left_index=True, right_index=True)
price_df = df[::-1]


***
## Strategy itself

In [53]:
def output_strategy(n_ls, data, output_sort=False):

    # Cumulative Returns for the Past Week
    log_ret_df = np.log(1 + data.pct_change())
    cum_returns = log_ret_df.apply(lambda x: x.sum())

    if output_sort == True:
        print(cum_returns.sort_values(ascending=False))
    

    top_stocks = cum_returns.sort_values(ascending=False).index[:n_ls].values
    bottom_stocks = cum_returns.sort_values(ascending=True).index[:n_ls].values

    print(f'Cryptos to long this week: {top_stocks}')
    print(f'Cryptos to short this week: {bottom_stocks}')



In [54]:
output_strategy(3, price_df, output_sort=True)

XRP      0.567636
SOL      0.500550
MATIC    0.192509
ADA      0.191358
AVAX     0.125376
DOGE     0.113979
LTC      0.103969
TRX      0.084832
BCH      0.080871
SHIB     0.068220
DOT      0.062336
BNB      0.044561
ETH      0.032845
TON     -0.005143
BTC     -0.005744
dtype: float64
Cryptos to long this week: ['XRP' 'SOL' 'MATIC']
Cryptos to short this week: ['BTC' 'TON' 'ETH']
